## Setups and configs

In [1]:
!pip install pandas

pyspark 2.3.1 requires py4j==0.10.7, which is not installed.
mkl-random 1.0.1 requires cython, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install textblob

pyspark 2.3.1 requires py4j==0.10.7, which is not installed.
mkl-random 1.0.1 requires cython, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [32]:
!pip install swifter

    100% |████████████████████████████████| 430kB 1.6MB/s ta 0:00:011
    100% |████████████████████████████████| 675kB 1.8MB/s ta 0:00:011
    100% |████████████████████████████████| 51kB 19.3MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 18.6MB/s ta 0:00:01
    100% |████████████████████████████████| 501kB 20.9MB/s ta 0:00:01
    100% |████████████████████████████████| 317kB 17.3MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 23.5MB/s ta 0:00:01
    100% |████████████████████████████████| 276kB 23.5MB/s ta 0:00:01
  Running setup.py bdist_wheel for swifter ... done
  Stored in directory: /root/.cache/pip/wheels/36/9e/b5/989a490f6200d7377c2bc046cd27fa61f32f158bc8b8e258b7
  Running setup.py bdist_wheel for psutil ... done
  Stored in directory: /root/.cache/pip/wheels/e2/9d/ea/1913d16f19bb927c32197308dec69cd8d10b61be8f7e265524
  Running setup.py bdist_wheel for toolz ... done
  Stored in directory: /root/.cache/pip/wheels/f4/0c/f6/ce6b2d1aa459ee97cc3c

In [39]:
!pip install sklearn

    100% |████████████████████████████████| 5.3MB 6.0MB/s eta 0:00:01
    100% |████████████████████████████████| 31.2MB 1.0MB/s eta 0:00:01
  Running setup.py bdist_wheel for sklearn ... done
  Stored in directory: /root/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn
pyspark 2.3.1 requires py4j==0.10.7, which is not installed.
mkl-random 1.0.1 requires cython, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Recommendation with PySpark

## Load the data into a dataFrame

In [3]:
data = spark.read.csv('/user/recommend/Reviews.csv', header=True, mode="DROPMALFORMED")

In [4]:
data.printSchema()

root
 |-- Id: string (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- UserId: string (nullable = true)
 |-- ProfileName: string (nullable = true)
 |-- HelpfulnessNumerator: string (nullable = true)
 |-- HelpfulnessDenominator: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Text: string (nullable = true)



In [5]:
data.show(3)

+---+----------+--------------+-----------+--------------------+----------------------+-----+----------+--------------------+--------------------+
| Id| ProductId|        UserId|ProfileName|HelpfulnessNumerator|HelpfulnessDenominator|Score|      Time|             Summary|                Text|
+---+----------+--------------+-----------+--------------------+----------------------+-----+----------+--------------------+--------------------+
|  1|B001E4KFG0|A3SGXH7AUHU8GW| delmartian|                   1|                     1|    5|1303862400|Good Quality Dog ...|I have bought sev...|
|  2|B00813GRG4|A1D87F6ZCVE5NK|     dll pa|                   0|                     0|    1|1346976000|   Not as Advertised|"Product arrived ...|
|  4|B000UA0QIQ|A395BORC6FGVXV|       Karl|                   3|                     3|    2|1307923200|      Cough Medicine|If you are lookin...|
+---+----------+--------------+-----------+--------------------+----------------------+-----+----------+--------------

In [6]:
data.columns

['Id',
 'ProductId',
 'UserId',
 'ProfileName',
 'HelpfulnessNumerator',
 'HelpfulnessDenominator',
 'Score',
 'Time',
 'Summary',
 'Text']

Drop columns that are not needed

In [7]:
data = data.drop('ProfileName', 'Time')
data.printSchema()

root
 |-- Id: string (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- UserId: string (nullable = true)
 |-- HelpfulnessNumerator: string (nullable = true)
 |-- HelpfulnessDenominator: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Text: string (nullable = true)



We will drop the user id since we will be building a item based recommendation system

In [8]:
data = data.drop('UserId')

Check for null values

In [75]:
from pyspark.sql.functions import isnan, when, count, col, isnull, lit

data.select([count(when(isnull(c), c)).alias(c) for c in data.columns]).show()

+---+---------+-----+-----------+------+
| Id|ProductId|Score|Helpfulness|Remark|
+---+---------+-----+-----------+------+
|  0|        0|    0|          0|     0|
+---+---------+-----+-----------+------+



Get the helpfulness ratio from the cols and add it to the dataframe

In [10]:
helpfulness = data['HelpfulnessNumerator'] / (data['HelpfulnessNumerator'] + data['HelpfulnessDenominator'])
data = data.withColumn('Helpfulness', helpfulness)

In [11]:
data = data.drop('HelpfulnessNumerator', 'HelpfulnessDenominator')
data.printSchema()

root
 |-- Id: string (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Text: string (nullable = true)
 |-- Helpfulness: double (nullable = true)



In [12]:
data = data.na.fill(0.0, 'Helpfulness')

## Feature selection

Append the `summary` and `text` column to a new feature and drop them

In [13]:
from pyspark.sql.functions import concat, lit

remark = concat(data['Summary'], lit(" "), data['Text'])
data = data.withColumn('Remark', remark)

In [14]:
data = data.drop('Summary', 'Text')
data.printSchema()

root
 |-- Id: string (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Helpfulness: double (nullable = false)
 |-- Remark: string (nullable = true)



In [15]:
data.select('Remark').show(10)

+--------------------+
|              Remark|
+--------------------+
|Good Quality Dog ...|
|Not as Advertised...|
|Cough Medicine If...|
|Great taffy Great...|
|Nice Taffy I got ...|
|Great!  Just as g...|
|Wonderful, tasty ...|
|Yay Barley Right ...|
|Healthy Dog Food ...|
|The Best Hot Sauc...|
+--------------------+
only showing top 10 rows



Convert the `Remark` feature into its corresponding sentiment polarity. Due to the diversity of words and context observed in the column word vectors will be too sparse and may not give good results

In [23]:
from textblob import TextBlob 

def to_sentiment(text):
    from textblob import TextBlob
    sen = TextBlob(text).sentiment.polarity
    return sen

convert to pandas and process

In [33]:
import swifter

remarks = data.toPandas()['Remark']
remarks = remarks.swifter.apply(to_sentiment)

Pandas Apply: 100%|██████████| 503209/503209 [08:50<00:00, 948.54it/s] 


In [88]:
new = data.toPandas()
new['Sentiment'] = remarks

,Id,ProductId,Score,Helpfulness,Remark,Sentiment
0,1,B001E4KFG0,5,0.5,Good Quality Dog Food I have bought several of...,0.485714
1,2,B00813GRG4,1,0.0,"Not as Advertised ""Product arrived labeled as ...",-0.0333333
2,4,B000UA0QIQ,2,0.5,Cough Medicine If you are looking for the secr...,0.166667
3,5,B006K2ZZ7K,5,0.0,Great taffy Great taffy at a great price. The...,0.546667
4,6,B006K2ZZ7K,4,0.0,Nice Taffy I got a wild hair for taffy and ord...,0.291667


In [90]:
new.drop(['Remark'], axis=1, inplace=True)

In [91]:
new_data = sqlContext.createDataFrame(new)
new_data.show(3)

+---+----------+-----+-----------+--------------------+
| Id| ProductId|Score|Helpfulness|           Sentiment|
+---+----------+-----+-----------+--------------------+
|  1|B001E4KFG0|    5|        0.5|  0.4857142857142857|
|  2|B00813GRG4|    1|        0.0|-0.03333333333333...|
|  4|B000UA0QIQ|    2|        0.5| 0.16666666666666666|
+---+----------+-----+-----------+--------------------+
only showing top 3 rows



Write the current state of dataframe as csv back to the hdfs

In [97]:
new_data.write.csv('/user/recommend/Reviews_new2.csv', header=True)

In [94]:
new_data.describe().show()

+-------+------------------+--------------------+-----------------+-------------------+-------------------+
|summary|                Id|           ProductId|            Score|        Helpfulness|          Sentiment|
+-------+------------------+--------------------+-----------------+-------------------+-------------------+
|  count|            503209|              503209|           503209|             503209|             503209|
|   mean| 284524.4813169081| 6.642618498597183E9|4.215810925480268|0.20846667140794187|0.27674306063227166|
| stddev|164106.34224156503|2.1005649557905338E9|1.294689191294415| 0.2346928634057244|  0.236348700194211|
|    min|                 1|          0006641040|                1|                0.0|               -1.0|
|    max|             99999|          B009WVB40S|                5|               0.75|                1.0|
+-------+------------------+--------------------+-----------------+-------------------+-------------------+



## Reload data

In [1]:
data_new = spark.read.csv('/user/recommend/Reviews_new2.csv', header=True, mode="DROPMALFORMED")
data_new.show(2)

+------+----------+-----+-----------+-------------------+
|    Id| ProductId|Score|Helpfulness|          Sentiment|
+------+----------+-----+-----------+-------------------+
|285151|B001E5E3JY|    5|        0.5|0.13819444444444448|
|285152|B001E5E3JY|    5|        0.5|0.35833333333333334|
+------+----------+-----+-----------+-------------------+
only showing top 2 rows



Calculate unreliability factor

In [2]:
from pyspark.sql.functions import stddev

unreliability = data_new.groupBy('ProductId').agg(stddev('Score').alias('Unreliability'))
colname = unreliability.columns[1]

unreliability = unreliability.na.fill(0.0, colname)
unreliability.show(2)

+----------+------------------+
| ProductId|     Unreliability|
+----------+------------------+
|B001E5E35S|0.5773502691896258|
|B0064KOSYE|               0.0|
+----------+------------------+
only showing top 2 rows



Get all the other features

In [3]:
from pyspark.sql.functions import avg, count

product_rating = data_new.groupBy('ProductId').agg(avg('Score').alias('Score'))
product_helpful = data_new.groupBy('ProductId').agg(avg('Helpfulness').alias('Helpfulness'))
product_sentiment = data_new.groupBy('ProductId').agg(avg('Sentiment').alias('Sentiment'))
product_rating_count = data_new.groupBy('ProductId').agg(count('Score').alias('Count'))

products = data_new.select('Productid').distinct()

Combine all the features into a new dataframe

In [4]:
data_model = products.join(product_rating_count, 'ProductId').join(product_rating, 'ProductId')\
.join(product_helpful, 'ProductId').join(product_sentiment, 'ProductId')\
.join(unreliability, 'ProductId')

In [5]:
data_model.show(5)

+----------+-----+-----+-------------------+--------------------+------------------+
| Productid|Count|Score|        Helpfulness|           Sentiment|     Unreliability|
+----------+-----+-----+-------------------+--------------------+------------------+
|B001E5E35S|    4|  4.5|0.20833333333333331|  0.3258531746031746|0.5773502691896258|
|B0064KOSYE|    1|  1.0|                0.0|-0.21388888888888888|               0.0|
|B0001NBGHM|    8| 3.75|0.26805555555555555|  0.2815917936230436|1.8322507626258084|
|B001BTJNNE|    1|  5.0|                0.0| 0.34920634920634924|               0.0|
|B004KZF39O|    1|  5.0|                0.5| 0.42346938775510207|               0.0|
+----------+-----+-----+-------------------+--------------------+------------------+
only showing top 5 rows



Normalize our data

In [6]:
def normalize(values):
    mn = values.min()
    mx = values.max()
    return 10.0 / (mx - mn) * (values - mx) + 10

In [7]:
mins, maxs = [], []
cols = data_model.columns[1:]

for col in cols:
    mn = data_model.agg({col: 'min'}).collect()[0][0]
    mins.append(mn)
    
for col in cols:
    mx = data_model.agg({col: 'max'}).collect()[0][0]
    maxs.append(mx)

mn, mx = min(mins), max(maxs) # get the min and max values of all the values in dataframe

def normalize(values):
    return 10.0 / (mx - mn) * (values - mx) + 10

In [20]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

udf_norm = udf(normalize, FloatType())

columns = data_model.columns[1:]
print(columns)



# data_model.withColumn('norm_count', udf_norm(data_model['Count'])).show(3)
for col in columns:
    data_model = data_model.withColumn(col, udf_norm(data_model[col]))
    
data_model.show(5)

['Count', 'Score', 'Helpfulness', 'Sentiment', 'Unreliability']
+----------+----------+-----------+-----------+-----------+-------------+
| Productid|     Count|      Score|Helpfulness|  Sentiment|Unreliability|
+----------+----------+-----------+-----------+-----------+-------------+
|B001E5E35S|0.05733945|  0.0630734|0.013857034|0.015204738|  0.018088879|
|B0064KOSYE|0.02293578| 0.02293578| 0.01146789|0.009015036|   0.01146789|
|B0001NBGHM|0.10321101|0.054472476|0.014541921|0.014697154|  0.032479938|
|B001BTJNNE|0.02293578| 0.06880734| 0.01146789| 0.01547255|   0.01146789|
|B004KZF39O|0.02293578| 0.06880734|0.017201835| 0.01632419|   0.01146789|
+----------+----------+-----------+-----------+-----------+-------------+
only showing top 5 rows



Checkpointing the dataframe. I'm basically paranoid

In [21]:
data_model.write.csv('/user/recommend/Reviews_norm.csv', header=True)

In [28]:
data_model = spark.read.csv('/user/recommend/Reviews_norm.csv', header=True, mode="DROPMALFORMED")

## Modeling the recommender

In [30]:
data_model = data_model.toPandas()

In [36]:
data_model.set_index('Productid', inplace=True)

In [37]:
data_model.head()

,Count,Score,Helpfulness,Sentiment,Unreliability
Productid,,,,,
B00019MN28,0.02293578,0.06880734,0.01146789,0.01827695,0.01146789
B0018CLVFM,0.03440367,0.06880734,0.01146789,0.012123198,0.01146789
B0001B8VRW,0.02293578,0.05733945,0.017201835,0.01425521,0.01146789
B002LSI1SW,0.2293578,0.064582326,0.014156534,0.014298305,0.024266867
B001UUVKQM,0.02293578,0.06880734,0.017201835,0.014393888,0.01146789


In [40]:
from sklearn.neighbors import KNeighborsClassifier

In [41]:
engine = KNeighborsClassifier(n_neighbors=20)

data_points = data_model.values
labels = data_model.index.values

print('Data: ', data_points)
print('Labels: ', labels)

engine.fit(data_points, labels)

Data:  [['0.02293578' '0.06880734' '0.01146789' '0.01827695' '0.01146789']
 ['0.03440367' '0.06880734' '0.01146789' '0.012123198' '0.01146789']
 ['0.02293578' '0.05733945' '0.017201835' '0.01425521' '0.01146789']
 ...
 ['0.02293578' '0.04587156' '0.01146789' '0.010841251' '0.01146789']
 ['0.06880734' '0.059633028' '0.01146789' '0.015051605' '0.031982277']
 ['0.02293578' '0.06880734' '0.01146789' '0.0163513' '0.01146789']]
Labels:  ['B00019MN28' 'B0018CLVFM' 'B0001B8VRW' ... 'B007FSAN9S' 'B0049BUWH2'
 'B003EMQDVG']


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=20, p=2,
           weights='uniform')

Let's make some recommendations

In [42]:
product_id = 'B0098WV8F2'
product_data = data_model.loc[product_id].values

recommended_products = engine.kneighbors(X=[product_data], n_neighbors=20, return_distance=False)

products_list = []
for prod in recommended_products:
    products_list.append(data_model.iloc[prod].index)
    
print('Recommended Products: ')
print(products_list)

Recommended Products: 
[Index(['B0098WV8F2', 'B000VK08OC', 'B004E4CCSQ', 'B000FMZO8G', 'B000FMZO90',
       'B004U49QU2', 'B0012NUVN0', 'B0012V1G0Y', 'B0009YJ4CW', 'B0045XE32E',
       'B005BRHVD6', 'B004YV80O4', 'B004E4EBMG', 'B004WTHCO2', 'B001LQCOIS',
       'B001P3PR54', 'B0009F3POY', 'B0083T6HC0', 'B0001ES9FI', 'B000OQ2DL4'],
      dtype='object', name='Productid')]
